In [ ]:
#|default_exp utils.sanitizer

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#|export
import pandas as pd
import numpy as np
import multiprocessing as mp
from tqdm import tqdm
from rdkit import Chem, rdBase
from rdkit.Chem import rdchem, rdmolops, SanitizeMol
from chembl_structure_pipeline import checker, standardizer
from rdkit.Chem.FilterCatalog import *
from rdkit.Chem.MolStandardize.rdMolStandardize import LargestFragmentChooser
from rdkit.Chem.SaltRemover import SaltRemover

[09:37:26] Initializing Normalizer


# Standardization

## Helper functions

In [ ]:
#|export
rdBase.DisableLog('rdApp.error')
rdBase.DisableLog('rdApp.info')
_saltremover = SaltRemover()
_unwanted = Chem.MolFromSmarts('[!#1!#5!#6!#7!#8!#9!#15!#16!#17!#35!#53]')

In [ ]:
#|export
def convert_smiles(mol, sanitize=False):
    if isinstance(mol, str):
        try:
            mol = Chem.MolFromSmiles(mol, sanitize=sanitize)
            return mol
        except:
            return None
    elif isinstance(mol, rdchem.Mol):
        return mol
    
def mol_to_inchi(mol):
    """Converts a rdchem.Mol object into InCHI keys"""
    mol = convert_smiles(mol, sanitize=True)
    inchi = Chem.MolToInchiKey(mol)
    return inchi   

def build_inchi_dict(smiles_list):
    if smiles_list is None:
        print("Please provide a valid list of molecules")
    inchi_dict = {mol_to_inchi(smi):smi for smi in smiles_list}
    return inchi_dict

def find_inchikey_duplicates(mol_list):
    from collections import Counter
    
    """Analyze a list of rdkit.Mol and identify InChIKey with more than molecule.

    Arguments:
    -------------------------------------------
    mol_list : List rdkit.Mol objects to analyze

    Returns:
    ------------------------------------------- 
    duplis : List of duplicates
    """
    inchi_dict = build_inchi_dict(mol_list)
    inchi_key_list = [mol_to_inchi(mol) for mol in mol_list]
    
    if len(inchi_key_list) == len(set(inchi_key_list)):
        return {}

    counts = dict(Counter(inchi_key_list))
    duplis = [inchi_dict[inchi] for inchi in inchi_dict.keys() if counts[inchi]>1]
    return duplis

In [ ]:
#|export
class MolCleaner:
    
    """Sanitize a molecular dataset
    
        Use static methods `MolCleaner.from_df or MolCleaner.from_csv` instead of accessing the \
        class directly.

        The sanitization process follows the steps implemented in the `chembl_structure_pipeline` package \
        (https://github.com/chembl/ChEMBL_Structure_Pipeline)
        
        1. Standardize unknown stereochemistry (Handled by the RDKit Mol file parser)
            i) Fix wiggly bonds on sp3 carbons - sets atoms and bonds marked as unknown stereo to no stereo
            ii) Fix wiggly bonds on double bonds – set double bond to crossed bond
        2. Clears S Group data from the mol file
        3. Kekulize the structure
        4. Remove H atoms (See the page on explicit Hs for more details)
        5. Normalization:
            Fix hypervalent nitro groups
            Fix KO to K+ O- and NaO to Na+ O- (Also add Li+ to this)
            Correct amides with N=COH
            Standardise sulphoxides to charge separated form
            Standardize diazonium N (atom :2 here: [*:1]-[N;X2:2]#[N;X1:3]>>[*:1]) to N+
            Ensure quaternary N is charged
            Ensure trivalent O ([*:1]=[O;X2;v3;+0:2]-[#6:3]) is charged
            Ensure trivalent S ([O:1]=[S;D2;+0:2]-[#6:3]) is charged
            Ensure halogen with no neighbors ([F,Cl,Br,I;X0;+0:1]) is charged
        6. The molecule is neutralized, if possible. See the page on neutralization rules for more details.
        7. Remove stereo from tartrate to simplify salt matching
        8. Normalise (straighten) triple bonds and allenes
        
        
        
        The curation steps in ChEMBL structure pipeline were augmented with additional steps to identify duplicated entries
        9. Find stereo centers
        10. Generate inchi keys
        11. Find duplicated SMILES. If the same SMILES is present multiple times, two outcomes are possible.
            i. The same compound (e.g. same ID and same SMILES)
            ii. Isomers with different SMILES, IDs and/or activities
            
            In case i), the compounds are merged by taking the median values of all numeric columns in the dataframe. 
            For case ii), the compounds are further classified as 'to merge' or 'to keep' depending on the activity values.
                a) Compounds are considered for mergining (to merge) if the difference in acvitities is less than 1log unit.
                b) Compounds are considered for keeping as individual entries (to keep) if the difference in activities is larger than 1log unit. In this case, the user can
                select which compound to keep - the one with highest or lowest activity.
        

    Parameters
    ------------------------------------------------------------------------------------------------------------
    
    ids : array-like
        An array of unique identifiers for molecules in a dataset.
            
    smiles : array-like
        An array of SMILES.
        
    Attributes
    ------------------------------------------------------------------------------------------------------------
    
    ids : array-like
        An array of unique identifiers for molecules in a dataset.
            
    smiles : array-like
        An array of SMILES.
        
    data : sanitized_data
        Sanitized dataset
        
    smiles_col : str
        A string representing the column of `data` with SMILES.
    
    id_col : str
        A string representing the column of `data` with IDs for each molecule.
       
    """
    def __init__(self, ids=None, smiles=None):


        self._ids = ids
        self._raw_smiles = smiles
        self._data = None
        self._id_col = None
        self._smiles_col = None
        self._act_col = None

    def __len__(self):
        return len(self._smiles)

    def __str__(self):
        return f'Raw dataset has {len(self.raw_smiles)} records\nThe first record is: {self.raw_smiles[0]}'

    def __getitem__(self, i):
        return self.raw_smiles[i]

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, i):
        self._data = i

    @property
    def raw_smiles(self):
        return self._raw_smiles

    @raw_smiles.setter
    def raw_smiles(self, i):
        self._raw_smiles = i

    @property
    def ids(self):
        return self._ids

    @ids.setter
    def ids(self, i):
        self._ids = i
        
    @property
    def act_col(self):
        return self._act_col

    @act_col.setter
    def act_col(self, i):
        self._act_col = i

    @property
    def id_col(self):
        return self._id_col

    @id_col.setter
    def id_col(self, i):
        self._id_col = i

    @property
    def smiles_col(self):
        return self._smiles_col

    @smiles_col.setter
    def smiles_col(self, i):
        self._smiles_col = i

    @classmethod
    def process_mol(cls, mol):
        """Fully process one molecule"""
        mol = convert_smiles(mol)
        try:
            # Remove salts and molecules with unwanted elements (See _unwanted definition above)
            mol = remove_unwanted(normalize_mol(mol))


            # Check if mixture is present - standard advice is to remove all mixtures
            mol = get_mixture(mol)

            if isinstance(mol, rdchem.Mol):
                return Chem.MolToSmiles(mol)
        except:
            return None

    @classmethod
    def process_smiles_list(cls,smiles_list):
        """Sanitize list of SMILES"""

        sanitized_smiles = map(cls.process_mol, tqdm(smiles_list, desc='Sanitizing SMILES', position=0, leave=True))

        return sanitized_smiles

    @classmethod
    def from_list(cls, smiles_list):
        
        """Factory method to process a collection of SMILES

        Arguments
        ------------------------------------------------------------------------------------------------------------

            smiles : A List, Array, or any Iterable (except strings)
                SMILES ready for sanitization

        Returns
            processed_mols : `pandas.DataFrame`

                A sanitized version of the `df` including all molecules. If sanitization fails for a molecule, \
                the original SMILES is kept but `processed_smiles` field will be NaN

        """
        act_col = 'Target'
        id_col = 'ID'
        smiles_col = 'SMILES'

        df = pd.DataFrame({smiles_col:smiles_list, 
                           id_col:[f'mol{idx}' for idx in range(len(smiles_list))],
                          act_col:[None]})


        return cls.from_df(df, 
                           smiles_col=smiles_col,
                           id_col=id_col, 
                           act_col=act_col)


    @classmethod
    def from_df(cls,
                df: pd.DataFrame, 
                smiles_col: str,
                act_col: str,
                id_col: str,
                keep:str='first'):
        
        """Factory method to process a `pandas.DataFrame`

        Arguments
        ------------------------------------------------------------------------------------------------------------

            df : pd.DataFrame
                A pandas Dataframe with molecular data for sanitization.

            smiles_col : str
                The name of the column with SMILES for each molecule.

            id_col : str
                The name of the column with IDs for each molecule. If None, generates random IDs.

        Returns
            processed_mols : `pandas.DataFrame`

                A sanitized version of the `df` including all molecules. If sanitization fails for a molecule,
                the original SMILES is kept but `processed_smiles` field will be NaN

        """     
        

        
        cls.data = df.copy()
        cls.data.reset_index(drop=True,inplace=True)
        cls.id_col = id_col
        cls.smiles_col = smiles_col
        cls.act_col = act_col

        cls.data[id_col] = cls.data[id_col].astype(str)

        cls.ids, cls.raw_smiles = cls.data[[id_col, smiles_col]].values.T



        return cls.sanitize_dataset(smiles=cls.raw_smiles,
                                    id_col=cls.id_col,
                                    act_col=cls.act_col,
                                    keep=keep)

    @classmethod
    def from_csv(cls,
                 data_path: str,
                 smiles_col: str,
                 act_col:str,
                id_col: str,
                 sep: str = ',',
                keep:str='first'):
        
        """Factory method to process a CSV file

        Arguments
        ------------------------------------------------------------------------------------------------------------

            data_path : str
                Path to CSV file

            smiles_col : str
                The name of the column with SMILES for each molecule.

            id_col : str
                The name of the column with IDs for each molecule. 
                
            id_col : str
                The name of the column with stereochemistry information for each molecule. 
 
        Returns
            processed_mols : `pandas.DataFrame`
            

                A sanitized version of the `df` including all molecules. If sanitization fails for a molecule,
                the original SMILES is kept but `processed_smiles` field will be NaN       
        
        """
        return cls.from_df(pd.read_csv(data_path, sep=sep), 
                           smiles_col=smiles_col,
                           id_col=id_col, 
                           act_col=act_col,
                           keep=keep)


    @classmethod
    def sanitize_dataset(cls,
                         smiles=None,
                         id_col:str=None,
                         smiles_col:str=None,
                         act_col:str=None,
                         keep:str=None):
        
        """Sanitize a dataset and returns ids, original SMILES and processed SMILES"""
        
        with mp.Pool() as mp_pool:
            sanitized_smiles = list(tqdm(mp_pool.imap(cls.process_mol, cls.raw_smiles),total=len(cls.raw_smiles)))
        #sanitized_smiles = cls.process_smiles_list(cls.raw_smiles)
 
        cls.data.insert(loc=1, column='processed_smiles', value=list(sanitized_smiles))
    

        cls.data.dropna(subset=['processed_smiles',cls.act_col],inplace=True)
        cls.data.reset_index(drop=True,inplace=True)
        
        # Remove duplicates
        cls.data = process_duplicates(cls.data,
                                            smiles_col='processed_smiles',
                                            act_col=cls.act_col,
                                            id_col=cls.id_col)

        return cls.data

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()

Converted MolClearner.ipynb.
